Analisi esplorativa:
	•	distribuzioni
	•	correlazioni meteo → PV
	•	pattern giornalieri
	•	differenze estate/inverno
	•	eventuali outliers


In [ ]:
# Cell 1: import e setup base

import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("default")
sns.set()

SEED = 42
np.random.seed(SEED)

# Aggiungo il path a src/ per importare data_loading.py
PROJECT_ROOT = Path("..").resolve()
SRC_DIR = PROJECT_ROOT / "src"
sys.path.append(str(SRC_DIR))

from data_loading import load_pv_data, load_weather_data, merge_pv_weather

In [ ]:
# Cell 2: carico i dataset PV e meteo

pv_df = load_pv_data()
wx_df = load_weather_data()

print("PV dataset:")
display(pv_df.head())
print(pv_df.info())

print("\nWeather dataset:")
display(wx_df.head())
print(wx_df.info())

In [ ]:
# Cell 3: esploro dimensioni e range temporali

print(f"PV shape: {pv_df.shape}")
print(f"WX shape: {wx_df.shape}")

print("\nRange temporale PV:")
print("  from:", pv_df.index.min())
print("  to  :", pv_df.index.max())

print("\nRange temporale WX:")
print("  from:", wx_df.index.min())
print("  to  :", wx_df.index.max())

# Tentativo di inferire la frequenza (non sempre funziona ma è indicativo)
print("\nFrequenza PV (infer_freq):", pd.infer_freq(pv_df.index))
print("Frequenza WX (infer_freq):", pd.infer_freq(wx_df.index))

In [ ]:
# Cell 4: unisco PV e meteo in un unico DataFrame

df = merge_pv_weather(pv_df, wx_df, how="inner")

print("Merged dataset shape:", df.shape)
display(df.head())

print("\nInfo merged dataset:")
print(df.info())

In [ ]:
# Cell 5: analisi missing values

missing_ratio = df.isna().mean().sort_values(ascending=False)
print("Percentuale di NaN per colonna:")
display(missing_ratio.to_frame("na_ratio"))

# Buchi temporali
expected_range = pd.date_range(df.index.min(), df.index.max(), freq="H")
print("\nNumero timestamp attesi (H):", len(expected_range))
print("Numero timestamp reali      :", len(df.index))

missing_timestamps = expected_range.difference(df.index)
print("Numero di timestamp mancanti:", len(missing_timestamps))

if len(missing_timestamps) > 0:
    print("Esempio di timestamp mancanti:", missing_timestamps[:10])

In [ ]:
# Cell 6: andamento della produzione PV nel tempo

target_col = "pv_power"

plt.figure(figsize=(15, 4))
df[target_col].plot()
plt.title("Produzione PV nel tempo (intero periodo)")
plt.ylabel("Potenza [kW] (circa)")  # aggiusta unità se necessario
plt.tight_layout()
plt.show()

# Zoom su una settimana
start = df.index.min()
end = start + pd.Timedelta(days=7)

plt.figure(figsize=(15, 4))
df.loc[start:end, target_col].plot()
plt.title("Produzione PV - prima settimana")
plt.ylabel("Potenza")
plt.tight_layout()
plt.show()

In [ ]:
# Cell 7: relazioni tra PV e alcune feature meteo

# tolgo la colonna max_kwp se presente
feature_cols = [c for c in df.columns if c not in ["pv_power", "max_kwp"]]

print("Numero di feature meteo:", len(feature_cols))
print(feature_cols)

# Matrice di correlazione (solo numeriche)
corr = df[["pv_power"] + feature_cols].select_dtypes(include=["number"]).corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr, cmap="coolwarm", center=0)
plt.title("Matrice di correlazione (PV + features numeriche)")
plt.tight_layout()
plt.show()

# Scatter veloce PV vs Ghi (Global horizontal irradiance), se presente
if "Ghi" in df.columns:
    sample = df.sample(min(5000, len(df)), random_state=SEED)
    plt.figure(figsize=(6, 5))
    sns.scatterplot(data=sample, x="Ghi", y="pv_power", alpha=0.3)
    plt.title("pv_power vs Ghi")
    plt.tight_layout()
    plt.show()

In [ ]:
# Cell 8: pattern per ora del giorno e per mese

df_eda = df.copy()
df_eda["hour"] = df_eda.index.hour
df_eda["month"] = df_eda.index.month

# Profilo medio giornaliero
hourly_profile = df_eda.groupby("hour")["pv_power"].mean()

plt.figure(figsize=(10, 4))
hourly_profile.plot(kind="bar")
plt.title("Profilo medio giornaliero della produzione PV")
plt.xlabel("Ora del giorno")
plt.ylabel("PV media")
plt.tight_layout()
plt.show()

# Profilo medio mensile
monthly_profile = df_eda.groupby("month")["pv_power"].mean()

plt.figure(figsize=(10, 4))
monthly_profile.plot(kind="bar")
plt.title("Profilo medio mensile della produzione PV")
plt.xlabel("Mese")
plt.ylabel("PV media")
plt.tight_layout()
plt.show()

In [ ]:
# Cell 9: salvo il dataset merged grezzo in processed_data

PROCESSED_DIR = PROJECT_ROOT / "processed_data"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

merged_path_parquet = PROCESSED_DIR / "merged_raw.parquet"
merged_path_csv = PROCESSED_DIR / "merged_raw.csv"

df.to_parquet(merged_path_parquet)
df.to_csv(merged_path_csv)

print("Salvato merged_raw.parquet e merged_raw.csv in", PROCESSED_DIR)